In [1]:
import os
import json
import numpy as np

# Strategy:
# Extract pos_ner + embeddings & rolling embeddings
# Flag bad ner, use POS to get the numbers and potential other bad stuff
# Project embeddings & rolling embeddings, cosine sim with all other clusters ?
# Then rules for numbers

text_path = '../example_conversations/english/labelled_text_conv_1.json'

with open(text_path, 'r') as f:
    labelled_data = json.load(f)

unlabelled_data = [list(i.keys())[0] for i in labelled_data]


In [2]:
from huggingface_hub import notebook_login, login

with open('hf_token.txt', 'r') as f:
    token = f.read()

login(token)

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

SMALL_MODEL = False

if SMALL_MODEL:
    MODEL_ID = 'google/gemma-3-1b-it'
else:
    MODEL_ID = 'google/gemma-3-12b-it'

dtype = torch.float32

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map={'': 'cpu'}, torch_dtype=dtype)

model.eval()

model.config.use_cache = True
device = model.device
print(device)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

cpu


In [4]:
with open('../prompt2.txt', 'r') as f:
    prompt = f.read()

rolled_window_data = []
win_down, win_up = -5, 5
for idx, word in enumerate(unlabelled_data):
    start, end = idx + win_down, idx + win_up
    start = max(0, start)
    end = min(end, len(unlabelled_data))

    rolled_window_data.append((' '.join(unlabelled_data[start:end]), word, idx))

def make_suffix(sentence: str, word_: str) -> str:
    return f'Sentence: {sentence}\nWord: {word_}\nAnswer:'

In [5]:
# Get KV cache
with torch.inference_mode():
    pref_inputs = tokenizer(prompt, return_tensors='pt').to(device)
    pref_out = model(**pref_inputs, use_cache=True)
    prompt_kv = pref_out.past_key_values
    past_len = pref_inputs.input_ids.size(1)

In [6]:
def continue_from_cache(prefix_kv, suffix, max_new_tokens=2, greedy=True):

    def forward_pass(input_ids, kv):
        past_len = kv[0][0].shape[-2]
        pos_ids = torch.arange(past_len, past_len + input_ids.size(1), device=device).unsqueeze(0)
        out = model(input_ids=input_ids,
                    attention_mask=torch.ones_like(input_ids),
                    position_ids=pos_ids,
                    past_key_values=kv,
                    use_cache=True)
        return out

    def get_next_token(logits):
        if greedy:
            return logits[:, -1, :].argmax(dim=-1, keepdim=True)

        return torch.distributions.Categorical(logits=logits[:, -1, :]).sample().unsqueeze(0)


    with torch.inference_mode():
        suf = tokenizer(suffix, return_tensors='pt').to(device)

        out = forward_pass(suf.input_ids, prefix_kv)

        next_token = get_next_token(out.logits)

        kv = out.past_key_values
        generated = [next_token]

        for _ in range(max_new_tokens - 1):

            out = forward_pass(next_token, kv)

            next_token = get_next_token(out.logits)

            generated.append(next_token)
            kv = out.past_key_values

    return tokenizer.decode(torch.cat(generated, dim=1)[0], skip_special_tokens=True)

In [7]:
suffix_ = make_suffix(rolled_window_data[0][0], rolled_window_data[0][1])

output = continue_from_cache(prefix_kv=prompt_kv, suffix=suffix_, max_new_tokens=5)

In [8]:
output

', and///'

In [10]:
tokens = tokenizer(prompt, return_tensors='pt')
out = model(**tokens)
tokenizer.decode(out.logits)


KeyboardInterrupt



In [ ]:
tokenizer.decode(out.logits)